In [2]:
import io
import pickle
from pathlib import Path

import numpy as np

In [3]:
results_path = Path(
    "../save/results/condmdi_random_joints/condsamples000750000__imputation_benchmark_clip_T=40_CI=0_CRG=0_KGP=1.0_seed10_person_throwing_a_ball/results.npy"
)
results = np.load(results_path, allow_pickle=True).item()

In [4]:
results.keys()

dict_keys(['sample', 'motion', 'text', 'lengths', 'num_samples', 'num_repetitions', 'observed_motion', 'observed_mask', 'pre_y', 'pre_xz', 'pre_rot'])

In [5]:
test_motion = results["motion"][0, 0, :, :, :10]  # 10 frames of first motion
test_motion = test_motion.transpose(2, 0, 1)
test_frame = test_motion[0]

print(f"Test Motion Shape: {test_motion.shape} using `{test_motion.dtype}`")

Test Motion Shape: (10, 22, 3) using `float32`


In [6]:
print("Using `np.save`:")

file_buffer = io.BytesIO()
np.save(file_buffer, test_frame)
print("Single Frame:\t", file_buffer.__sizeof__())

file_buffer = io.BytesIO()
np.save(file_buffer, test_motion)
print(f"{len(test_motion)} Frames:\t", file_buffer.__sizeof__())

Using `np.save`:
Single Frame:	 490
10 Frames:	 2866


In [7]:
print("Using `pickle.dumps`:")

test_pickle = pickle.dumps(test_frame)
print("Single Frame:\t", test_pickle.__sizeof__())

test_pickle = pickle.dumps(test_motion)
print(f"{len(test_motion)} Frames:\t", test_pickle.__sizeof__())

Using `pickle.dumps`:
Single Frame:	 449
10 Frames:	 2827


## Pack/Unpack motion

In [24]:
masked_motion = test_motion.copy()
masked_motion[:, 16:, :] = np.nan

In [25]:
keyframes = {
    0: masked_motion[0].tolist(),
    # 1: masked_motion[1].tolist(),
    # 2: masked_motion[2].tolist(),
    3: masked_motion[3].tolist(),
    # 4: masked_motion[4].tolist(),
    # 5: masked_motion[5].tolist(),
    # 6: masked_motion[6].tolist(),
}


In [37]:
input_frames = np.max(list(keyframes.keys()))
rot_motion = np.zeros((input_frames + 1, 22, 3))
joint_mask = np.ones((input_frames + 1, 22, 1), dtype=bool)
for frame, pose in keyframes.items():
    _pose = np.array(pose)
    _pose_mask = ~np.isnan(_pose)
    rot_motion[frame, _pose_mask] = _pose[_pose_mask]
    joint_mask[frame] = _pose_mask.all(axis=-1).reshape(-1, 1)

# Extract root position and mask
root_pos = rot_motion[:, 0].copy()
root_mask = joint_mask[:, 0].copy()
rot_motion = rot_motion[:, 1:]
joint_mask = joint_mask[:, 1:]


In [38]:
root_pos

array([[ 0.        ,  0.96076506,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [-0.00181807,  0.95707607,  0.02317366]])